In [1]:
using Zygote, ReverseDiff, ForwardDiff, LinearAlgebra

In [91]:
"""
u = Ax + some constant col vector
"""
function f_test_1(A, x)
    u = A*x[2:end] .+ x[1]
    return u
end

"""
alloc inside
"""
function f_test_2(A, x)
    u = Vector{Float64}(undef, length(x)-1)
    u .= A*x[2:end] .+ x[1]
    return u
end

"""
non-alloc ver
"""
function f_test_3!(u, A, x)
    u .= A*x[2:end] .+ x[1]
end


"""
unrolled loop ver (nonalloc)
"""
function f_test_4!(u, A, x)
    row_idx = 1:size(A)[1]; col_idx = 1:size(A)[2]
    for j ∈ col_idx
        for i ∈ row_idx
            u[i] += A[i,j]*x[j+1]
        end
    end
    for i ∈ row_idx
        u[i] += x[1]
    end
end

"""
= _4 but allocates & returns the output explicitly
"""
function f_test_4b(A, x)
    row_size = size(A)[1]
    row_idx = 1:row_size; col_idx = 1:size(A)[2]
    u = Vector{Float64}(undef, row_size); fill!(u, 0.)
    for j ∈ col_idx
        for i ∈ row_idx
            u[i] += A[i,j]*x[j+1]
        end
    end
    for i ∈ row_idx
        u[i] += x[1]
    end
    return u
end

# derivative examples:
J_f_1(A, x) = Zygote.jacobian(x -> f_test_1(A, x), x)
J_f_2(A, x) = Zygote.jacobian(x -> f_test_2(A, x), x)
J_f_3(u, A, x) = Zygote.jacobian(x -> f_test_3!(u, A, x), x)

J_f_3 (generic function with 1 method)

In [95]:
# test feval:
A = Matrix{Float64}(LinearAlgebra.I, 5, 5)
u = Vector{Float64}(undef, 5); fill!(u, 0.)
x = ones(6)
f_test_3!(u, A, x)
display(u)

fill!(u, 0.)
f_test_4!(u, A, x)
display(u)

5-element Vector{Float64}:
 2.0
 2.0
 2.0
 2.0
 2.0

5-element Vector{Float64}:
 2.0
 2.0
 2.0
 2.0
 2.0

In [86]:
# test FD:
A = Matrix{Real}(LinearAlgebra.I, 5, 5) 
u = Vector{Real}(undef, 5); fill!(u, 0.)
ForwardDiff.jacobian(x -> f_test_3!(u, A, x), x)
# only works when arrays' datatype = Real (which allocates to memoryu ⟹ slow)

5×6 Matrix{Real}:
 1.0  1.0  0.0  0.0  0.0  0.0
 1.0  0.0  1.0  0.0  0.0  0.0
 1.0  0.0  0.0  1.0  0.0  0.0
 1.0  0.0  0.0  0.0  1.0  0.0
 1.0  0.0  0.0  0.0  0.0  1.0

In [87]:
# test RD:
A = Matrix{Float64}(LinearAlgebra.I, 5, 5) 
u = Vector{Float64}(undef, 5); fill!(u, 0.)
#Zygote.jacobian(x -> f_test_3!(u, A, x), x)
Zygote.jacobian(x -> f_test_1(A, x), x)
# only works for f_test_1

([1.0 1.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0; … ; 1.0 0.0 … 1.0 0.0; 1.0 0.0 … 0.0 1.0],)

In [101]:
# test [RD FD] on [_4 _4b]:
fill!(u, 0.)
#ForwardDiff.jacobian(x -> f_test_4b(A, x), x)
Zygote.jacobian(x -> f_test_4!(u, A, x), x)
#=
_4:
    - FD: output not detected...
    - RD: idem
_4b:
    - FD: asks the output to be Real
    - RD: usual 'mutating array' error
=#

LoadError: ArgumentError: jacobian expected a function which returns an array, or a scalar, got Nothing

In [2]:
# closure for tapes by @Steffen:
function comp(A)
    function f(x) 
        return A*x[2:end] .+ x[1] 
    end
    return f 
end

A = Matrix{Float64}(LinearAlgebra.I, 5,5)
x = ones(6)

f_tape = ReverseDiff.JacobianTape( comp(A), x)
comp_f_tape = ReverseDiff.compile(f_tape)

res = Array{Float64}(undef, (size(A)[1], length(x)))

ReverseDiff.jacobian!(res, comp_f_tape, x)

5×6 Matrix{Float64}:
 1.0  1.0  0.0  0.0  0.0  0.0
 1.0  0.0  1.0  0.0  0.0  0.0
 1.0  0.0  0.0  1.0  0.0  0.0
 1.0  0.0  0.0  0.0  1.0  0.0
 1.0  0.0  0.0  0.0  0.0  1.0